In [5]:
from __future__ import print_function

import pandas as pd
import numpy as np
import requests
from threading import Thread
import glob

In [6]:
files = glob.glob('./surveys/*.csv')
files

['./surveys\\1st Trust Survey - (3).csv',
 './surveys\\2nd Trust Survey (5,non-adaptive,every mission debrief).csv',
 './surveys\\2nd Trust Survey (6,adaptive, every mission debrief).csv',
 './surveys\\Demographic Survey (1).csv',
 './surveys\\End of Experiment Survey (20).csv',
 './surveys\\IPIP Survey (2).csv',
 './surveys\\NASA-TLX Rating Scale (4,every mission debrief).csv',
 './surveys\\NASA-TLX Weighted Scale (21).csv']

In [9]:
class upload_surveys:
    #requires a survey folder
    #requires a debrief folder
    def __init__(self,path="./surveys/*.csv"):
        self.path = path
        self.survey_column_name = "Survey Name"
        self.question_column_name = "Question"
        self.type_column_name = "Type"
        self.offered_answers_name = "Official Answers"
        iURL = 'http://ec2-52-24-126-225.us-west-2.compute.amazonaws.com:81/sql-temp/insert'
        threadpool = []
        for fname in glob.glob(self.path):
            #self.upload_survey(fname, iURL)
            t = Thread(target= self.upload_survey, args=(fname,iURL,))
            threadpool.append(t)
        for t in threadpool:
            t.start()
        for t in threadpool:
            t.join()
        
    def send_query(self,query,URL):
        iData = {
                    'sql': query,
                    'database': 'dbsurveys'
                }
        
        #print (iData)
        req = requests.post(url=URL, data=iData)
        #if "200" in req:
         #   print(query)
        #print(req)
        #print(req.text)
    def remove_survey (self,survey_id):
        iData = {
                    'survey_id': survey_id
                }
        URL = 'http://ec2-52-24-126-225.us-west-2.compute.amazonaws.com:81/survey/remove-survey'
        #print (iData)
        req = requests.post(url=URL, data=iData)

    def upload_survey(self,path,iURL):
        
        data = pd.read_csv(path)
        print(data)
        #print (data.iloc[0][0])
        #print (data.iloc[2][0])
        survey_name = data.iloc[0][0]
        survey_id = int(data.iloc[2][0])
        self.remove_survey(survey_id)
        print(data)
        print(list(data)[0])
        data.drop(["Survey Name"], axis=1,inplace=True)
        data.dropna(axis=0, how='all', inplace=True)
        #will craete a new entry if it does not already exists
        survey_query =("INSERT INTO dbsurveys.survey (id, description)" + 
                        "SELECT * FROM (SELECT '{0}', '{1}') AS tmp " + 
                        "WHERE NOT EXISTS (" + 
                        "SELECT description FROM dbsurveys.survey " +
                        "WHERE description = '{1}') LIMIT 1;")
        survey_query = survey_query.format(survey_id,survey_name)

        question_query = "INSERT INTO dbsurveys.question (id, type, question_text) VALUE "
        offered_answer_query = "INSERT INTO dbsurveys.offered_answer (id, offered_answer_text) VALUE"
        survey_question_query = "INSERT INTO dbsurveys.survey_question (survey_id, question_id) VALUE "
        question_offered_answer_query =  "INSERT INTO dbsurveys.question_answer (question_id, offered_answer_id) VALUE "
        
        answers = []
        #print (data)
        for i, row in data.iterrows():
            index = i + survey_id*100
            
            
            question_text = str(row[self.question_column_name]).replace("'", "\\'")
            question_type = str(row[self.type_column_name]).replace("'", "\\'")
            
            if question_text == "":
#                 question_query = question_query[:-1]
#                 offered_answer_query = offered_answer_query[:-1]
#                 survey_question_query = survey_question_query[:-1]
#                 question_offered_answer_query = question_offered_answer_query[:-1]
                continue
            offered_answer = ""
            if(pd.isnull(row[self.offered_answers_name])):
                offered_answer_text = ""
                offered_answer_query += "('{0}', '{1}')".format(index, offered_answer_text)
            else:
                offered_answer_text = str(row[self.offered_answers_name]).replace("'", "\\'")
                offered_answer_query += "('{0}','{1}')".format(index, offered_answer_text)
            question_query += "('{0}', '{1}', '{2}')".format(index,question_type, question_text)
            survey_question_query += "('{0}','{1}')".format(survey_id,index)
            question_offered_answer_query += "('{0}','{1}')".format(index,index)
            #if index > 5:
             #       break

            if i < len(data.index) - 1:
                question_query += ","
                offered_answer_query += ","
                survey_question_query += ","
                question_offered_answer_query += ","
          
           
        question_query += ";\n\n"
        offered_answer_query += ";\n\n"
        survey_question_query += ";\n\n"
        question_offered_answer_query += ";\n\n"

        #breaks post request for me
        query = survey_query+ question_query + offered_answer_query + survey_question_query + question_offered_answer_query    
        print ("path = "+ path + "\n\n" + query)
        self.send_query(survey_query , iURL)
        self.send_query(question_query , iURL)
        self.send_query(offered_answer_query , iURL)
        self.send_query(survey_question_query , iURL)
        self.send_query(question_offered_answer_query , iURL)
        
        
        #req = requests.post(url=iURL, data=iData)

In [10]:
path = "./surveys/*.csv"
upload_surveys()


        Survey Name                                           Question   Type  \
0  Mission Survey 1  We are not only interested in assessing your p...  Intro   
1             Order  How much mental and perceptual activity was re...    TLX   
2                 4  How much physical activity was required (e.g. ...    TLX   
3  Adaptivity level  How much time pressure did you feel due to the...    TLX   
4                 2  How successful do you think you were in accomp...    TLX   
5    Last Edited By  How hard did you have to work (mentally and ph...    TLX   
6     Falisha Kanji  How insecure, discouraged, irritated, stressed...    TLX   
7       Last Edited  Taking into account all of the above scales, w...    TLX   
8          01/31/18  Thank you for participating! Please click "nex...  Outro   

  Official Answers  
0              NaN  
1         Low|High  
2         Low|High  
3         Low|High  
4        Poor|Good  
5         Low|High  
6         Low|High  
7         Low|High  


<__main__.upload_surveys instance at 0x000000000B58B088>

In [7]:
data

NameError: name 'data' is not defined

In [30]:
survey_id = 4
iData = {
            'survey_id': survey_id
        }
URL = 'http://ec2-52-24-126-225.us-west-2.compute.amazonaws.com:81/survey/remove-survey'
#print (iData)
req = requests.post(url=URL, data=iData)